In [1]:
from liberate import fhe
from liberate.fhe import presets

In [2]:
params = presets.params["silver"]
print(params)
engine = fhe.ckks_engine(verbose=True, **params)

sk = engine.create_secret_key()
pk = engine.create_public_key(sk)

gk = engine.create_galois_key(sk)
evk = engine.create_evk(sk)

{'logN': 15, 'num_special_primes': 2, 'devices': [0]}
[2023-11-14 13:23:34.579195] I have received the context:


I have received inputs:
        buffer_bit_length		= 62
        scale_bits			= 40
        logN				= 15
        N				= 32,768
        Number of special primes	= 2
        Number of scales		= 16
        Cache folder			= '/home/hanyul/.pyenv/versions/liberate/lib/python3.11/site-packages/liberate/fhe/cache/resources'
        Security bits			= 128
        Quantum security model		= post_quantum
        Security sampling distribution	= uniform
        Number of message bits		= 60
        In total I will be using '821' bits out of available maximum '829' bits.
        And is it secured?		= True
My RNS primes are [1099510054913, 1099515691009, 1099507695617, 1099516280833, 1099506515969, 1099520606209, 1099504549889, 1099523555329, 1099503894529, 1099527946241, 1099503370241, 1099529060353, 1099498258433, 1099531223041, 1099469684737, 1099532009473, 1152921504606584833, 11529215045

# Auto level & data types

In [3]:
a = engine.example(-255, 255)
b = engine.example(-255, 255)
c = engine.example(-255, 255)

delta = 0.2

result_ = ((a * b + 3) * delta - c) - b


print("====="*17)
print("|\toperation ((a * b + 3) * delta - c) - b : absmax error\t\t\t   |")
print("-----"*17)
for level in range(engine.num_levels-4):

    cta = engine.encorypt(a, pk, level)
    ctb = engine.encorypt(b, pk, level)

    result = engine.mult(cta, ctb, evk)
    result = engine.add(result, 3)
    result = engine.mult(result, delta, evk)
    result = engine.sub(result, c)
    result = engine.sub(result, ctb)

    copied_ct = engine.clone(result)

    result = engine.decrode(result, sk)

    print(f"| level from : {level:2d} |  {engine.absmax_error(result, result_):.15e} | level to : {copied_ct.level:2d}  |")
print("====="*17)

|	operation ((a * b + 3) * delta - c) - b : absmax error			   |
-------------------------------------------------------------------------------------
| level from :  0 |  2.204086740675848e-06+2.705879524000920e-06j | level to :  2  |
| level from :  1 |  2.097303877235390e-06+2.246369831482298e-06j | level to :  3  |
| level from :  2 |  3.597437171265483e-06+2.374439645791426e-06j | level to :  4  |
| level from :  3 |  2.086162567138672e-06+2.184888217016123e-06j | level to :  5  |
| level from :  4 |  2.281074557686225e-06+2.570393007772509e-06j | level to :  6  |
| level from :  5 |  2.044369921350153e-06+2.261212102894206e-06j | level to :  7  |
| level from :  6 |  2.634261818457162e-06+2.079497789964080e-06j | level to :  8  |
| level from :  7 |  2.259252141811885e-06+2.697204763535410e-06j | level to :  9  |
| level from :  8 |  2.344535459997132e-06+2.504127678548684e-06j | level to : 10  |
| level from :  9 |  3.522122824506368e-06+2.134642272721976e-06j | level to : 11  |


In [4]:
a = engine.example(-255, 255)
b = engine.example(-255, 255)

ct0 = engine.encorypt(a, pk, 1)
ct1 = engine.encorypt(b, pk, 10)

evk = engine.create_evk(sk)
print("====="*15)
print("|\t\ttest mult, add, sub\t\t\t\t\t|")
print("-----"*15)
for level0 in range(engine.num_levels-1):
    ct0 = engine.encorypt(a, pk, level=level0)

    for level1 in range(engine.num_levels-1):
        ct1 = engine.encorypt(b, pk, level=level1)

        ctm = engine.mult(ct0, ct1, evk)
        cta = engine.add(ct0, ct1)
        cts = engine.sub(ct0, ct1)

        mm = engine.decrode(ctm, sk)
        ma = engine.decrode(cta, sk)
        ms = engine.decrode(cts, sk)

        mul = a*b
        add = a+b
        sub = a-b

        print(f"|\t{level0:2d}, {level1:2d}, 'mult' |  {engine.absmax_error(mm, mul):.15e}\t|")
        print(f"|\t{level0:2d}, {level1:2d}, 'add'  |  {engine.absmax_error(ma, add):.15e}\t|")
        print(f"|\t{level0:2d}, {level1:2d}, 'sub'  |  {engine.absmax_error(ms, sub):.15e}\t|")

        print("-----"*15)
    print("====="*15)
print("====="*15)

|		test mult, add, sub					|
---------------------------------------------------------------------------
|	 0,  0, 'mult' |  1.190981856780127e-05+1.380272442474961e-05j	|
|	 0,  0, 'add'  |  3.400657533347839e-10+9.433108516532229e-09j	|
|	 0,  0, 'sub'  |  3.063860276597552e-10+9.565710001879779e-09j	|
---------------------------------------------------------------------------
|	 0,  1, 'mult' |  1.748374779708683e-05+1.242163853021339e-05j	|
|	 0,  1, 'add'  |  3.378622182026447e-08+3.576184326448129e-08j	|
|	 0,  1, 'sub'  |  3.387893343642645e-08+3.589232733247627e-08j	|
---------------------------------------------------------------------------
|	 0,  2, 'mult' |  1.320396040682681e-05+1.346567296423018e-05j	|
|	 0,  2, 'add'  |  3.374614720996760e-08+3.576951712602749e-08j	|
|	 0,  2, 'sub'  |  3.389556013644324e-08+3.591983954720490e-08j	|
---------------------------------------------------------------------------
|	 0,  3, 'mult' |  1.299397263210267e-05+1.585004793014377e-05j

|	 7, 10, 'mult' |  3.590806227293797e-05+2.963767951769114e-05j	|
|	 7, 10, 'add'  |  1.153785547103325e-07+7.731895834695024e-08j	|
|	 7, 10, 'sub'  |  1.153458555336329e-07+7.717983407928841e-08j	|
---------------------------------------------------------------------------
|	 7, 11, 'mult' |  4.223964060656726e-05+2.963273891509743e-05j	|
|	 7, 11, 'add'  |  1.152261006609478e-07+7.721307326846727e-08j	|
|	 7, 11, 'sub'  |  1.152435658013928e-07+7.703971505179652e-08j	|
---------------------------------------------------------------------------
|	 7, 12, 'mult' |  4.071845614816993e-05+2.551922807469964e-05j	|
|	 7, 12, 'add'  |  1.154693904936721e-07+7.730176321274485e-08j	|
|	 7, 12, 'sub'  |  1.151668129750760e-07+7.725412842773949e-08j	|
---------------------------------------------------------------------------
|	 7, 13, 'mult' |  2.866332943085581e-05+2.680327452253550e-05j	|
|	 7, 13, 'add'  |  1.151430524259922e-07+7.714892547028285e-08j	|
|	 7, 13, 'sub'  |  1.1516719666815

|	 8,  9, 'mult' |  3.500485036056489e-05+3.011877379321959e-05j	|
|	 8,  9, 'add'  |  6.867887236694514e-08+7.720856842752255e-08j	|
|	 8,  9, 'sub'  |  6.866180513043219e-08+7.719413019913191e-08j	|
---------------------------------------------------------------------------
|	 8, 10, 'mult' |  3.124489012407139e-05+2.819849032675847e-05j	|
|	 8, 10, 'add'  |  6.879048441987834e-08+7.744171170998015e-08j	|
|	 8, 10, 'sub'  |  6.841968058779457e-08+7.722371719864896e-08j	|
---------------------------------------------------------------------------
|	 8, 11, 'mult' |  4.418080061441287e-05+2.842870981112355e-05j	|
|	 8, 11, 'add'  |  6.855299261587788e-08+7.746251640128321e-08j	|
|	 8, 11, 'sub'  |  6.867998081361293e-08+7.718838901382696e-08j	|
---------------------------------------------------------------------------
|	 8, 12, 'mult' |  2.939005207736045e-05+2.777389090624638e-05j	|
|	 8, 12, 'add'  |  6.864544843665499e-08+7.727200568297121e-08j	|
|	 8, 12, 'sub'  |  6.8561405441869

|	 9,  8, 'mult' |  3.063809708692133e-05+4.661464481614530e-05j	|
|	 9,  8, 'add'  |  7.763958365103463e-08+7.792866085765127e-08j	|
|	 9,  8, 'sub'  |  7.750113439897177e-08+7.790350764480536e-08j	|
---------------------------------------------------------------------------
|	 9,  9, 'mult' |  2.667723310878500e-05+2.729336847551167e-05j	|
|	 9,  9, 'add'  |  4.311004886403680e-10+1.899235257951659e-08j	|
|	 9,  9, 'sub'  |  4.319815616327105e-10+1.893940293484775e-08j	|
---------------------------------------------------------------------------
|	 9, 10, 'mult' |  3.126061346847564e-05+3.454887360021530e-05j	|
|	 9, 10, 'add'  |  6.425176479751826e-08+7.437121496423060e-08j	|
|	 9, 10, 'sub'  |  6.444703615215985e-08+7.442446303684846e-08j	|
---------------------------------------------------------------------------
|	 9, 11, 'mult' |  3.866553015541285e-05+2.857480694729020e-05j	|
|	 9, 11, 'add'  |  6.438898481064825e-08+7.436420901285601e-08j	|
|	 9, 11, 'sub'  |  6.4327466020586

|	10,  7, 'mult' |  3.326795194880106e-05+3.190177085343748e-05j	|
|	10,  7, 'add'  |  8.101301318674814e-08+7.216755193439894e-08j	|
|	10,  7, 'sub'  |  8.096846215721598e-08+7.247144284683316e-08j	|
---------------------------------------------------------------------------
|	10,  8, 'mult' |  3.256032505305484e-05+2.700871118577197e-05j	|
|	10,  8, 'add'  |  9.226192787537002e-08+8.525624650701502e-08j	|
|	10,  8, 'sub'  |  9.249134791389224e-08+8.517645255778916e-08j	|
---------------------------------------------------------------------------
|	10,  9, 'mult' |  3.192443546140566e-05+3.790936898440123e-05j	|
|	10,  9, 'add'  |  7.796961654094048e-08+6.625000992244168e-08j	|
|	10,  9, 'sub'  |  7.789779488120985e-08+6.617989356527687e-08j	|
---------------------------------------------------------------------------
|	10, 10, 'mult' |  2.622378087835386e-05+2.034877252299339e-05j	|
|	10, 10, 'add'  |  4.676792286772979e-10+1.912979996632203e-08j	|
|	10, 10, 'sub'  |  4.7328008179192

...

|	30, 19, 'mult' |  3.235286931158043e-05+3.357221430633217e-05j	|
|	30, 19, 'add'  |  6.401688779078540e-08+7.225017029099945e-08j	|
|	30, 19, 'sub'  |  6.409121056094591e-08+7.248644351420808e-08j	|
---------------------------------------------------------------------------
|	30, 20, 'mult' |  2.707716339500621e-05+2.994191163452342e-05j	|
|	30, 20, 'add'  |  9.308840276389674e-08+8.020833774935454e-08j	|
|	30, 20, 'sub'  |  9.324584482328646e-08+8.027130604659760e-08j	|
---------------------------------------------------------------------------
|	30, 21, 'mult' |  4.631125557352789e-05+3.285519051132724e-05j	|
|	30, 21, 'add'  |  7.980253258210723e-08+8.568835596634017e-08j	|
|	30, 21, 'sub'  |  7.990362949072960e-08+8.572470733270166e-08j	|
---------------------------------------------------------------------------
|	30, 22, 'mult' |  2.854409467545338e-05+3.026630656677298e-05j	|
|	30, 22, 'add'  |  7.719884820289735e-08+7.486698905267986e-08j	|
|	30, 22, 'sub'  |  7.7003051046631

|	31, 22, 'mult' |  2.774067252175882e-05+2.795817999867722e-05j	|
|	31, 22, 'add'  |  7.215373898361577e-08+8.110356475299341e-08j	|
|	31, 22, 'sub'  |  7.251026090671076e-08+8.086278313612638e-08j	|
---------------------------------------------------------------------------
|	31, 23, 'mult' |  3.554766226443462e-05+3.526044747559354e-05j	|
|	31, 23, 'add'  |  8.856073918650509e-08+8.051276267906360e-08j	|
|	31, 23, 'sub'  |  8.852484256749449e-08+8.050758992794727e-08j	|
---------------------------------------------------------------------------
|	31, 24, 'mult' |  2.815260813804343e-05+3.051238309126347e-05j	|
|	31, 24, 'add'  |  7.965461890080405e-08+6.774882876925403e-08j	|
|	31, 24, 'sub'  |  7.969113369199476e-08+6.764071258658078e-08j	|
---------------------------------------------------------------------------
|	31, 25, 'mult' |  4.359488957561553e-05+3.153398574795574e-05j	|
|	31, 25, 'add'  |  8.159059916579281e-08+6.489489123850944e-08j	|
|	31, 25, 'sub'  |  8.1739699453464

|	32, 26, 'mult' |  3.688971992232837e-05+3.050638406421058e-05j	|
|	32, 26, 'add'  |  6.641502636739460e-08+7.521850875491509e-08j	|
|	32, 26, 'sub'  |  6.651112016697880e-08+7.526944045821438e-08j	|
---------------------------------------------------------------------------
|	32, 27, 'mult' |  2.913235221058130e-05+3.352024486957816e-05j	|
|	32, 27, 'add'  |  6.873676738905488e-08+6.604648206121055e-08j	|
|	32, 27, 'sub'  |  6.892497594890301e-08+6.619963954790364e-08j	|
---------------------------------------------------------------------------
|	32, 28, 'mult' |  3.424726310186088e-05+2.701679113670252e-05j	|
|	32, 28, 'add'  |  7.172309324232629e-08+7.469333240806009e-08j	|
|	32, 28, 'sub'  |  7.167832904997340e-08+7.458406514615490e-08j	|
---------------------------------------------------------------------------
|	32, 29, 'mult' |  3.107354859821498e-05+3.035331610590219e-05j	|
|	32, 29, 'add'  |  8.823764119370026e-08+8.528853889799848e-08j	|
|	32, 29, 'sub'  |  8.8364856765110

|	33, 32, 'mult' |  3.316345100756735e-05+2.997127558046486e-05j	|
|	33, 32, 'add'  |  7.627835429957486e-08+8.072424151350788e-08j	|
|	33, 32, 'sub'  |  7.641021237958512e-08+8.071820190025392e-08j	|
---------------------------------------------------------------------------
|	33, 33, 'mult' |  2.178344220737927e-05+2.523795410525054e-05j	|
|	33, 33, 'add'  |  4.029061528854072e-10+1.896449930427480e-08j	|
|	33, 33, 'sub'  |  4.900186922895955e-10+1.883807954072836e-08j	|
---------------------------------------------------------------------------
|	33, 34, 'mult' |  3.142642526654527e-05+3.242039383621886e-05j	|
|	33, 34, 'add'  |  8.723236177843319e-08+7.219870212793467e-08j	|
|	33, 34, 'sub'  |  8.715119292901363e-08+7.203773577657557e-08j	|
---------------------------------------------------------------------------
|	33, 35, 'mult' |  2.838121145032346e-05+2.726940783759346e-05j	|
|	33, 35, 'add'  |  8.725204025950006e-08+7.201663265732350e-08j	|
|	33, 35, 'sub'  |  8.6944737631711

|	35,  6, 'mult' |  3.768418173422106e-05+2.930207051576872e-05j	|
|	35,  6, 'add'  |  6.575598376912239e-08+7.358578102412139e-08j	|
|	35,  6, 'sub'  |  6.553173648171651e-08+7.367276566583314e-08j	|
---------------------------------------------------------------------------
|	35,  7, 'mult' |  3.647789344540797e-05+3.084451600443572e-05j	|
|	35,  7, 'add'  |  6.842978450549708e-08+7.904822041382431e-08j	|
|	35,  7, 'sub'  |  6.837575483586988e-08+7.904927201707324e-08j	|
---------------------------------------------------------------------------
|	35,  8, 'mult' |  3.000755532411858e-05+4.717832041478687e-05j	|
|	35,  8, 'add'  |  9.286057434110262e-08+7.100119603364874e-08j	|
|	35,  8, 'sub'  |  9.301604109168693e-08+7.088860343174019e-08j	|
---------------------------------------------------------------------------
|	35,  9, 'mult' |  3.119454049738124e-05+3.043162359972484e-05j	|
|	35,  9, 'add'  |  1.006386298740836e-07+8.643826276966138e-08j	|
|	35,  9, 'sub'  |  1.0038282027835